In [ ]:
#%load_ext soup
import requests
from bs4 import BeautifulSoup
import pandas as pd
from os import path

In [ ]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"

In [ ]:
r = requests.get(url)
isa_form = BeautifulSoup(r.text, 'html.parser')
#print(isa_form.prettify())

In [ ]:
request_codes = {}
for input in isa_form.find_all('select'):
    for element in input.find_all('option'):
        #print(element.text)
        request_codes[element.text] = (input['name'], element['value'])

In [ ]:
arguments = {}
arguments["ww_x_GPS"] = "-1"
arguments["ww_i_reportModel"] = "133685247"
arguments["ww_i_reportModelXsl"] = "133685270"
arguments[request_codes["Informatique"][0]] = request_codes["Informatique"][1]
#arguments[request_codes["2007-2008"][0]] = request_codes["2007-2008"][1]
arguments[request_codes["Bachelor semestre 1"][0]] = request_codes["Bachelor semestre 1"][1]
arguments["ww_x_HIVERETE"] = "null"

In [ ]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?"
#r = requests.get(url, params=arguments)

In [ ]:
for year in range(2007,2017):
    # fill in params request: year
    y = str(year)+"-"+str(year+1)
    arguments[request_codes[y][0]] = request_codes[y][1]
    for semester in [1,2,3,4]:
        # fill in params request: semester
        b = "Master semestre %d" % semester
        arguments[request_codes[b][0]] = request_codes[b][1]
        # do request and save to file
        filename = 'data/master_%d_%d.csv' % (year, semester)
        #if not path.isfile(filename):
        r = requests.get(url, params=arguments)
        #print(r.headers)
        #print(r.status_code)
        try:
            table = pd.read_html(r.text, flavor='lxml', skiprows=1, header=0)[0]
            print(year, semester)
            table = table.dropna(axis=1, how='all')
            table['year'] = year
            table['semester'] = semester
            table.to_csv(filename)
        except ValueError as e:
            pass
        

In [ ]:
for year in range(2007,2017):
    # fill in params request: year
    y = str(year)+"-"+str(year+1)
    arguments[request_codes[y][0]] = request_codes[y][1]
    for semester in ["automne","printemps"]:
        # fill in params request: semester
        b = "Projet Master %s" % semester
        arguments[request_codes[b][0]] = request_codes[b][1]
        # do request and save to file
        filename = 'data/projet_master_%d_%s.csv' % (year, semester)
        #if not path.isfile(filename):
        r = requests.get(url, params=arguments)
        #print(r.headers)
        #print(r.status_code)
        try:
            table = pd.read_html(r.text, flavor='lxml', skiprows=1, header=0)
            if(len(table) != 0):
                table[0] = table[0].dropna(axis=1, how='all')
                table[0]['year'] = year
                table[0]['semester'] = semester
                print(str(year)+" "+semester)
                table[0].to_csv(filename)
        except ValueError as e:
            pass

In [ ]:
# take a look at a small sample
pd1 = pd.read_csv("data/master_2007_1.csv")
pd1.head()

In [ ]:
pd2 = pd.read_csv("data/projet_master_2007_printemps.csv")
pd2.head()